In [1]:
import pandas as pd

df = pd.read_csv('./data/October_November_missions_full.csv')
df.rename(columns={'sub': 'user'}, inplace=True)
df['mission'] = df['kind'] + '_' + df['TARGET'].astype(str)

df = df[['user', 'mission', 'createdAt', 'kind', 'TARGET', 'performance']]
df = df.groupby('user').filter(lambda x: x['createdAt'].nunique() > 10)
df['createdAt'] = pd.to_datetime(df['createdAt']).dt.date

df['user'] = df['user'].astype('category').cat.codes
df['mission'] = df['mission'].astype('category')
df['missionID'] = df['mission'].cat.codes
df['kind'] = df['kind'].astype('category')

def reward(x):
    if x <= 1:
        return x
    return max(0, 2 - x**2)

df['reward'] = df['performance'].apply(reward)

df.sort_values(by=['createdAt', 'user'], inplace=True, ignore_index=True)
df

,user,mission,createdAt,kind,TARGET,performance,missionID,reward
0,2,streak_1,2024-10-02,streak,1,0.000000,30,0.0
1,2,quiz_2,2024-10-02,quiz,2,2.500000,25,0.0
2,2,episode_3,2024-10-02,episode,3,2.333333,14,0.0
3,2,activity_8,2024-10-02,activity,8,2.125000,10,0.0
4,2,activity_10,2024-10-02,activity,10,1.700000,3,0.0
...,...,...,...,...,...,...,...,...
121912,1194,quiz_3,2024-11-30,quiz,3,0.000000,26,0.0
121913,1194,streak_1,2024-11-30,streak,1,0.000000,30,0.0
121914,1196,activity_3,2024-11-30,activity,3,0.000000,5,0.0
121915,1196,episode_5,2024-11-30,episode,5,0.000000,16,0.0


In [7]:
n_users = df['user'].max() + 1
n_missions = df['missionID'].max() + 1

print(n_users, n_missions)

missions = df[['missionID', 'kind', 'TARGET']].drop_duplicates().set_index('missionID')
missions

1197 31


,kind,TARGET
missionID,,
30,streak,1
25,quiz,2
14,episode,3
10,activity,8
3,activity,10
19,exp,50
24,quiz,1
13,episode,2
11,activity,9


In [3]:
from src import contextual as ctx
from tqdm.auto import tqdm

def recommend(rank: list, missions: pd.DataFrame, n=1) -> list:
    ranked_missions = missions.loc[rank] # ranks missions
    top_missions = ranked_missions.groupby('kind', observed=True, sort=False).head(1) # selects the top mission of each kind
    return top_missions.index.tolist()[:n] # returns the top n missions

def replay(df: pd.DataFrame, policy: ctx.LinUCB):
    history = pd.DataFrame()
    for t, round in tqdm(df.groupby('createdAt'), leave=False):
        day_recs = []
        for u in tqdm(round['user'].unique(), leave=False):
            rank = policy.select(u)
            recs = [{'user': u, 'missionID': rec} for rec in recommend(rank, missions, n=3)]
            day_recs += recs

        actions = round.merge(pd.DataFrame(day_recs), on=['user', 'missionID'], how='inner')
        history = pd.concat((history, actions), ignore_index=True)
        policy.update(train_df=history, day=t)
            
    return history


def evaluate(policy) -> pd.DataFrame:
    rewards = replay(df[['user', 'missionID', 'createdAt', 'reward']], policy)
    rewards = rewards.groupby('createdAt')['reward'].sum().cumsum()

    return rewards

In [ ]:
import torch
import numpy

torch.manual_seed(0)
numpy.random.seed(0)

policies = {
    'LinUCB': ctx.LinUCB(n_missions, context_manager=ctx.ContextManager(n_users, n_missions)),
}

results = pd.concat([
    pd.concat({name: evaluate(policy) for name, policy in tqdm(policies.items(), leave=False)})
    for _ in tqdm(range(5))
], axis=1)

results
results.to_csv('./out/replay_results_baseline.csv', index=True)